# Grounding with Vertex AI Agent Builder

## Overview

[Grounding in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/ground-language-models) lets you use generative text models to generate content grounded in your own documents and data. This capability lets the model access information at runtime that goes beyond its training data. By grounding model responses in data stores within [Vertex AI Agent Builder](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction), LLMs that are grounded in data can produce more accurate, up-to-date, and relevant responses.

Grounding provides the following benefits:

- Reduces model hallucinations (instances where the model generates content that isn't factual)
- Anchors model responses to specific information, documents, and data sources
- Enhances the trustworthiness, accuracy, and applicability of the generated content

### Learning Objectives

In this notebook, you learn how to:

- Compare the results of ungrounded LLM responses with grounded LLM responses
- Create and use a data store in Vertex AI Search to ground responses in custom documents and data
- Generate LLM text and chat model responses grounded in Vertex AI Search results

This tutorial uses the following Google Cloud AI services and resources:

- Vertex AI
- Vertex AI Search and Conversation

The steps performed include:

- Configuring the LLM and prompt for various examples
- Sending example prompts to generative text and chat models in Vertex AI
- Setting up a data store in Vertex AI Search with your own data
- Sending example prompts with various levels of grounding (no grounding, data store grounding)

**Acknowledgement**  
This notebook is based on a notebook by [Holt Skinner](https://github.com/holtskinner) and [Kristopher Overholt](https://github.com/koverholt)

In [6]:
PROJECT_ID = !gcloud config list --format 'value(core.project)'
PROJECT_ID = PROJECT_ID[0]
REGION = "us-central1"

In [7]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

### Import libraries

In [10]:
import vertexai
from IPython.display import Markdown, display
from vertexai.language_models import (
    ChatModel,
    GroundingSource,
    TextGenerationModel,
)

Initialize the text and chat palm model from Vertex AI:

In [20]:
parameters = {
    "temperature": 0.5,  # Temperature controls the degree of randomness in token selection.
    "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
    "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
    "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
}

text_model = TextGenerationModel.from_pretrained("text-bison@002")
chat_model = ChatModel.from_pretrained("chat-bison@002")

## Grounding with custom documents and data

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the [results of a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest). You'll ask a question about a GoogleSQL query to create an [object table in BigQuery](https://cloud.google.com/bigquery/docs/object-table-introduction).

### Creating a data store in Vertex AI Search

Follow the steps below to create a data store in Vertex AI Search with sample data. In this example, you'll use a website-based data store that contains content from the Google Cloud website, including documentation.

1. In the Google Cloud console, go to the [Agent Builder](https://console.cloud.google.com/gen-app-builder/engines?_ga=2.175627042.1438957058.1715191726-353778574.1715191726) page.
2. Click **New** app.
3. In the Select app type pane, select **Search**.
4. If you are offered a choice of generic or media content, click **Generic**.
5. In order to create a website search app, make sure **Enterprise features** is turned on.
6. If you don't plan to use **Advanced LLM** features for this app, turn off the **Advanced LLM** features option.
7. In the **Your app name** field, enter a name for your app. Your app ID appears under the app name.
8. In the **External name** of your company or organization field, enter the company or organization name.
9. Select **global (Global)** as the location for your app, and then click Continue.
10. In the **Data stores** pane, click **Create new data store**.
11. In the Select a data source pane, select **Website URLs**.
12. Make sure that **Advanced website indexing** is turned off.
13. In the **Specify the websites** for your data store pane, in the Sites to include field, enter `cloud.google.com/*` and then click Continue.
14. In the **Configure your data store** pane, enter a name for your data store, and then click **Create**.
15. On the **Data stores** page, select your new data store, and then click **Create**.

Once you've created a data store, obtain the Data Store ID and input it below.

In [12]:
DATA_STORE_PROJECT_ID = PROJECT_ID  # @param {type:"string"}
DATA_STORE_REGION = "global"  # @param {type:"string"}
# Replace this with your data store ID from Vertex AI Search
DATA_STORE_ID = "gcp-documentation_1715192184974"  # @param {type:"string"}

Now you can ask a question about object tables in BigQuery and when to use them:

In [13]:
PROMPT = "When to use an object table in BigQuery?"

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [15]:
response = text_model.predict(
    PROMPT,
    **parameters,
)
print(f"Response from model without grounding:\n {response.text}")

Response from model without grounding:
  Object tables in BigQuery are used when you want to store and query data that has a complex structure, such as nested objects or arrays. Object tables are also useful when you want to store data that is not easily represented in a relational table, such as JSON or XML data.

Here are some specific cases where you might want to use an object table in BigQuery:

* To store data that has a complex structure, such as nested objects or arrays. For example, you could use an object table to store customer data, where each customer has a name, address, and list of orders.
* To store data that is not easily represented in a relational table, such as JSON or XML data. For example, you could use an object table to store data from a web service that returns JSON responses.
* To improve query performance on data that has a complex structure. Object tables can help to improve query performance by allowing you to filter and aggregate data based on the properti

### Text generation grounded in Vertex AI Search results

Now we can add the `tools` keyword arg with a grounding tool of `grounding.VertexAISearch()` to instruct the LLM to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [19]:
if DATA_STORE_ID and DATA_STORE_REGION:
    # Use Vertex AI Search data store
    grounding_source = GroundingSource.VertexAISearch(
        data_store_id=DATA_STORE_ID, location=DATA_STORE_REGION
    )
else:
    print("Please provide DATA_STORE_ID and DATA_STORE_REGION")

response = text_model.predict(
    PROMPT,
    grounding_source=grounding_source,
    **parameters,
)
print(f"Response from Model:\n{response.text}")
print(f"\n\nGrounding Metadata:\n{response.grounding_metadata}")

Response from Model:
 Object tables can be used in BigQuery in the following scenarios:

- To analyze unstructured data in Cloud Storage.
- To perform analysis with remote functions.
- To perform inference using BigQuery ML.


Grounding Metadata:
GroundingMetadata(citations=[GroundingCitation(start_index=37, end_index=99, url='https://cloud.google.com/bigquery/docs/object-table-introduction', title='Introduction to object tables | BigQuery | Google Cloud', license=None, publication_date=None), GroundingCitation(start_index=101, end_index=159, url='https://cloud.google.com/bigquery/docs/object-table-introduction', title='Introduction to object tables | BigQuery | Google Cloud', license=None, publication_date=None), GroundingCitation(start_index=161, end_index=219, url='https://cloud.google.com/bigquery/docs/object-table-introduction', title='Introduction to object tables | BigQuery | Google Cloud', license=None, publication_date=None)], search_queries=['When to use an object table in Bi

Note that the response without grounding only has limited information from the LLM about object tables in BigQuery that might not be accurate. Whereas the response that was grounded in Vertex AI Search results contains the most up to date information from the Google Cloud documentation about BigQuery, along with citations of the information.

## Grounded Chat Responses

You can also use grounding when working with chat models in Vertex AI. In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search and a data store in Vertex AI Search.

You'll ask a question about Vertex AI and a follow up question about managed datasets in Vertex AI:

In [21]:
PROMPT = "What are managed datasets in Vertex AI?"
PROMPT_FOLLOWUP = "What types of data can I use"

### Chat session without grounding

Start a chat session and send messages to the LLM with no grounding:

In [22]:
chat = chat_model.start_chat()

response = chat.send_message(PROMPT)
print(response.text)

response = chat.send_message(PROMPT_FOLLOWUP)
print(response.text)

 Managed datasets in Vertex AI are curated and maintained by Google. They provide a convenient way to access high-quality data for training and evaluating machine learning models. Managed datasets are available in a variety of domains, including healthcare, finance, and retail.

Here are some of the benefits of using managed datasets in Vertex AI:

* **High-quality data:** Managed datasets are curated by Google experts to ensure that they are accurate, complete, and consistent.
* **Convenient access:** Managed datasets are hosted in the cloud, so you can access them from anywhere with an internet connection.
* **Scalable:** Managed datasets can be scaled to meet the needs of your machine learning models.
* **Secure:** Managed datasets are protected by Google's robust security measures.

To learn more about managed datasets in Vertex AI, please visit the [documentation](https://cloud.google.com/vertex-ai/docs/datasets/overview).
 You can use a variety of data types with Vertex AI, inclu

### Chat session grounded in Vertex AI Search results

Now you can add the `grounding_source` keyword arg with a grounding source of `GroundingSource.VertexAISearch()` to instruct the chat model to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [24]:
chat = chat_model.start_chat()
grounding_source = GroundingSource.VertexAISearch(
    data_store_id=DATA_STORE_ID, location=DATA_STORE_REGION
)

response = chat.send_message(
    PROMPT,
    grounding_source=grounding_source,
)
print(response.text)
print(response.grounding_metadata)

response = chat.send_message(
    PROMPT_FOLLOWUP,
    grounding_source=grounding_source,
)
print(response.text)
print(response.grounding_metadata)

 Managed datasets in Vertex AI are used to provide the source data used to train AutoML and custom models.
GroundingMetadata(citations=[GroundingCitation(start_index=1, end_index=106, url='https://cloud.google.com/vertex-ai/docs/datasets/overview', title='Overview of creating managed datasets on Vertex AI | Google Cloud', license=None, publication_date=None)], search_queries=['Vertex AI managed datasets'])
 You can use the following types of data to train a model:

- Tabular data: This is data that is organized in rows and columns, like a spreadsheet.
- Image data: This is data that is represented as a collection of pixels, like a photograph.
- Video data: This is data that is represented as a sequence of images, like a movie.
- Text data: This is data that is represented as a sequence of characters, like a document.
GroundingMetadata(citations=[GroundingCitation(start_index=1, end_index=58, url='https://cloud.google.com/vertex-ai/docs/training-overview', title='Train and use your own 

Copyright 2024 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License